In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
import time
import json
import statistics
import seaborn as sns
import csv

# Comparison data for Hammersmith

- This file will look at alternative data for Hammersmith the ensure findings from API data were accurate
- This data has been verified and used by Air Quality England but has a high amount of NaN values compared to the other set of data
- The data still contains anomalous values
- This file will clean this data to be used in comparison with the API data

## Data cleaning

In [47]:
# Data had to been downloaded in separate csvs to cover the whole time period
hammersmith_21 = pd.read_csv('../data_unclean/Hammersmith_air_pollution_2021.csv')
hammersmith_22 = pd.read_csv('../data_unclean/Hammersmith_air_pollution_2022.csv')
hammersmith_23 = pd.read_csv('../data_unclean/Hammersmith_air_pollution_2023.csv')

In [48]:
hammersmith_21.head()

,Data download from AQ England website,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Status: P - Provisional data,P* - Ratified by Data Provider,R - Ratified Data,V - Validated,A - As supplied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hourly Data,Hourly Data,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre
4,End Date,End Time,PM10,Status/units,NO,Status/units,NO2,Status/units,NOXasNO2,Status/units,O3,Status/units,PM25,Status/units


In [49]:
hammersmith_21 = hammersmith_21.drop([0, 1, 2, 3])

hammersmith_21.columns = hammersmith_21.iloc[0]
hammersmith_21 = hammersmith_21.drop(hammersmith_21.index[0]).reset_index(drop=True)
hammersmith_21.columns = [
    "End Date", "End Time", "PM10", "PM10 Status/units",
    "NO", "NO Status/units", "NO2", "NO2 Status/units",
    "NOXasNO2", "NOXasNO2 Status/units", "O3", "O3 Status/units",
    "PM25", "PM25 Status/units"
]

In [50]:
hammersmith_21.head()

,End Date,End Time,PM10,PM10 Status/units,NO,NO Status/units,NO2,NO2 Status/units,NOXasNO2,NOXasNO2 Status/units,O3,O3 Status/units,PM25,PM25 Status/units
0,01/01/2021,01:00:00,27.5,R ugm-3 (Ref.eq),20.92937,R ugm-3,38.37475,R ugm-3,70.466,R ugm-3,16.81045,R ugm-3,24.4,R ugm-3 (BAM)
1,01/01/2021,02:00:00,61.7,R ugm-3 (Ref.eq),6.0364,R ugm-3,31.40525,R ugm-3,40.66093,R ugm-3,17.71516,R ugm-3,67.4,R ugm-3 (BAM)
2,01/01/2021,03:00:00,25.8,R ugm-3 (Ref.eq),3.36416,R ugm-3,21.27606,R ugm-3,26.43437,R ugm-3,30.83357,R ugm-3,24.4,R ugm-3 (BAM)
3,01/01/2021,04:00:00,14.2,R ugm-3 (Ref.eq),3.38563,R ugm-3,21.6773,R ugm-3,26.86853,R ugm-3,28.39881,R ugm-3,14.4,R ugm-3 (BAM)
4,01/01/2021,05:00:00,19.2,R ugm-3 (Ref.eq),3.0349,R ugm-3,20.14869,R ugm-3,24.80214,R ugm-3,29.4715,R ugm-3,19.4,R ugm-3 (BAM)


In [51]:
hammersmith_22.head()

,Data download from AQ England website,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Status: P - Provisional data,P* - Ratified by Data Provider,R - Ratified Data,V - Validated,A - As supplied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hourly Data,Hourly Data,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre,Hammersmith Town Centre
4,End Date,End Time,PM10,Status/units,NO,Status/units,NO2,Status/units,NOXasNO2,Status/units,O3,Status/units,PM25,Status/units


In [52]:
hammersmith_22 = hammersmith_22.drop([0, 1, 2, 3])

hammersmith_22.columns = hammersmith_22.iloc[0]
hammersmith_22 = hammersmith_22.drop(hammersmith_22.index[0]).reset_index(drop=True)
hammersmith_22.columns = [
    "End Date", "End Time", "PM10", "PM10 Status/units",
    "NO", "NO Status/units", "NO2", "NO2 Status/units",
    "NOXasNO2", "NOXasNO2 Status/units", "O3", "O3 Status/units",
    "PM25", "PM25 Status/units"
]

In [53]:
hammersmith_22.head()

,End Date,End Time,PM10,PM10 Status/units,NO,NO Status/units,NO2,NO2 Status/units,NOXasNO2,NOXasNO2 Status/units,O3,O3 Status/units,PM25,PM25 Status/units
0,01/01/2022,01:00:00,25,R ugm-3 (Ref.eq),12.51919,R ugm-3,14.0019,R ugm-3,33.19773,R ugm-3,24.50886,R ugm-3,17,R ugm-3 (BAM)
1,01/01/2022,02:00:00,20.8,R ugm-3 (Ref.eq),10.44011,R ugm-3,11.78449,R ugm-3,27.79244,R ugm-3,NaN,NaN,10,R ugm-3 (BAM)
2,01/01/2022,03:00:00,16.7,R ugm-3 (Ref.eq),7.9662,R ugm-3,9.57374,R ugm-3,21.7884,R ugm-3,26.60892,R ugm-3,9,R ugm-3 (BAM)
3,01/01/2022,04:00:00,15.8,R ugm-3 (Ref.eq),4.87049,R ugm-3,5.53982,R ugm-3,13.00781,R ugm-3,30.71437,R ugm-3,6,R ugm-3 (BAM)
4,01/01/2022,05:00:00,14.2,R ugm-3 (Ref.eq),5.93563,R ugm-3,8.68479,R ugm-3,17.78597,R ugm-3,33.89048,R ugm-3,3,R ugm-3 (BAM)


In [54]:
hammersmith_23.head()

,End Date,End Time,PM10,Status/units,NO,Status/units.1,NO2,Status/units.2,NOXasNO2,Status/units.3,O3,Status/units.4,PM25,Status/units.5,Unnamed: 14
0,01/01/2023,01:00:00,14.2,R ugm-3 (Ref.eq),2.76395,R ugm-3,6.13576,R ugm-3,10.37376,R ugm-3,75.08183,R ugm-3,9.0,R ugm-3 (BAM),NaN
1,01/01/2023,02:00:00,12.5,R ugm-3 (Ref.eq),2.99566,R ugm-3,7.11976,R ugm-3,11.71304,R ugm-3,NaN,NaN,7.0,R ugm-3 (BAM),NaN
2,01/01/2023,03:00:00,14.2,R ugm-3 (Ref.eq),2.59177,R ugm-3,7.22349,R ugm-3,11.19748,R ugm-3,74.64906,R ugm-3,6.0,R ugm-3 (BAM),NaN
3,01/01/2023,04:00:00,16.7,R ugm-3 (Ref.eq),3.53521,R ugm-3,5.82500,R ugm-3,11.24558,R ugm-3,77.11490,R ugm-3,13.0,R ugm-3 (BAM),NaN
4,01/01/2023,05:00:00,17.5,R ugm-3 (Ref.eq),1.55490,R ugm-3,6.46746,R ugm-3,8.85160,R ugm-3,76.83475,R ugm-3,10.0,R ugm-3 (BAM),NaN


In [55]:
hammersmith_23 = hammersmith_23.rename(columns={'Status/units':'PM10 Status/units','Status/units.1':'NO Status/units','Status/units.2':'NO2 Status/units',
                               'Status/units.3':'NOXasNO2 Status/units','Status/units.4':'O3 Status/units','Status/units.5':'PM25 Status/units'})

In [56]:
hammersmith_23 = hammersmith_23.drop(columns='Unnamed: 14')
hammersmith_23.columns

Index(['End Date', 'End Time', 'PM10', 'PM10 Status/units', 'NO',
       'NO Status/units', 'NO2', 'NO2 Status/units', 'NOXasNO2',
       'NOXasNO2 Status/units', 'O3', 'O3 Status/units', 'PM25',
       'PM25 Status/units'],
      dtype='object')

In [57]:
hammersmith_verified = pd.concat([hammersmith_21,hammersmith_22, hammersmith_23])
hammersmith_verified.columns

Index(['End Date', 'End Time', 'PM10', 'PM10 Status/units', 'NO',
       'NO Status/units', 'NO2', 'NO2 Status/units', 'NOXasNO2',
       'NOXasNO2 Status/units', 'O3', 'O3 Status/units', 'PM25',
       'PM25 Status/units'],
      dtype='object')

In [58]:
hammersmith_verified.shape

(26286, 14)

In [59]:
hammersmith_verified = hammersmith_verified.drop(columns=['PM10 Status/units', 'NO Status/units', 'NO2 Status/units',
                                                          'NOXasNO2 Status/units','O3 Status/units','PM25 Status/units'])
hammersmith_verified.columns

Index(['End Date', 'End Time', 'PM10', 'NO', 'NO2', 'NOXasNO2', 'O3', 'PM25'], dtype='object')

In [60]:
hammersmith_verified = hammersmith_verified.rename(columns = {'End Date':'date','End Time':'time'})
hammersmith_verified.head(30)

,date,time,PM10,NO,NO2,NOXasNO2,O3,PM25
0,01/01/2021,01:00:00,27.5,20.92937,38.37475,70.466,16.81045,24.4
1,01/01/2021,02:00:00,61.7,6.0364,31.40525,40.66093,17.71516,67.4
2,01/01/2021,03:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4
3,01/01/2021,04:00:00,14.2,3.38563,21.6773,26.86853,28.39881,14.4
4,01/01/2021,05:00:00,19.2,3.0349,20.14869,24.80214,29.4715,19.4
5,01/01/2021,06:00:00,20.8,3.82942,21.27712,27.14881,28.71812,22.4
6,01/01/2021,07:00:00,10.8,4.33762,17.10154,23.75246,30.36957,17.4
7,01/01/2021,08:00:00,12.5,4.44499,23.43834,30.25389,26.81722,20.4
8,01/01/2021,09:00:00,15,5.58665,23.81465,32.38073,26.66255,15.4
9,01/01/2021,10:00:00,20.8,6.01612,22.81541,32.03999,28.01963,19.4


In [61]:
nans = hammersmith_verified.isna().sum()
nans

# There are some nans in date and time so these must be removed

date           3
time           6
PM10        3404
NO          2133
NO2         2144
NOXasNO2    2144
O3          3425
PM25        2901
dtype: int64

In [62]:
hammersmith_verified = hammersmith_verified.dropna(subset=['date', 'time'])

In [63]:
nans = hammersmith_verified.isna().sum()
nans

date           0
time           0
PM10        3398
NO          2127
NO2         2138
NOXasNO2    2138
O3          3419
PM25        2895
dtype: int64

In [64]:
# Need to deal with nan vlaues in this data but need to do further cleaning first

In [65]:
# Some times are in incorrect format 24:00:00 so these need to be changed to perform strptime function

hammersmith_verified['time'] = ['23:59:59' if x == '24:00:00' else x for x in hammersmith_verified['time']]
hammersmith_verified.head(30)

,date,time,PM10,NO,NO2,NOXasNO2,O3,PM25
0,01/01/2021,01:00:00,27.5,20.92937,38.37475,70.466,16.81045,24.4
1,01/01/2021,02:00:00,61.7,6.0364,31.40525,40.66093,17.71516,67.4
2,01/01/2021,03:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4
3,01/01/2021,04:00:00,14.2,3.38563,21.6773,26.86853,28.39881,14.4
4,01/01/2021,05:00:00,19.2,3.0349,20.14869,24.80214,29.4715,19.4
5,01/01/2021,06:00:00,20.8,3.82942,21.27712,27.14881,28.71812,22.4
6,01/01/2021,07:00:00,10.8,4.33762,17.10154,23.75246,30.36957,17.4
7,01/01/2021,08:00:00,12.5,4.44499,23.43834,30.25389,26.81722,20.4
8,01/01/2021,09:00:00,15,5.58665,23.81465,32.38073,26.66255,15.4
9,01/01/2021,10:00:00,20.8,6.01612,22.81541,32.03999,28.01963,19.4


In [66]:
# create new date_time column

hammersmith_verified['date_time'] = hammersmith_verified['date'] + ' ' + hammersmith_verified['time']
hammersmith_verified.head()

,date,time,PM10,NO,NO2,NOXasNO2,O3,PM25,date_time
0,01/01/2021,01:00:00,27.5,20.92937,38.37475,70.466,16.81045,24.4,01/01/2021 01:00:00
1,01/01/2021,02:00:00,61.7,6.0364,31.40525,40.66093,17.71516,67.4,01/01/2021 02:00:00
2,01/01/2021,03:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4,01/01/2021 03:00:00
3,01/01/2021,04:00:00,14.2,3.38563,21.6773,26.86853,28.39881,14.4,01/01/2021 04:00:00
4,01/01/2021,05:00:00,19.2,3.0349,20.14869,24.80214,29.4715,19.4,01/01/2021 05:00:00


In [67]:
# format date time

hammersmith_verified['date_time'] = hammersmith_verified['date_time'].apply(lambda row: datetime.strptime(row, '%d/%m/%Y %H:%M:%S'))


In [68]:
hammersmith_verified.head()

,date,time,PM10,NO,NO2,NOXasNO2,O3,PM25,date_time
0,01/01/2021,01:00:00,27.5,20.92937,38.37475,70.466,16.81045,24.4,2021-01-01 01:00:00
1,01/01/2021,02:00:00,61.7,6.0364,31.40525,40.66093,17.71516,67.4,2021-01-01 02:00:00
2,01/01/2021,03:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4,2021-01-01 03:00:00
3,01/01/2021,04:00:00,14.2,3.38563,21.6773,26.86853,28.39881,14.4,2021-01-01 04:00:00
4,01/01/2021,05:00:00,19.2,3.0349,20.14869,24.80214,29.4715,19.4,2021-01-01 05:00:00


In [69]:
# time was end time so subtract 1 hour from each time to match original data

hammersmith_verified['date_time'] = pd.to_datetime(hammersmith_verified['date_time'])
hammersmith_verified['date_time'] = hammersmith_verified['date_time'].apply(lambda dt: dt - timedelta(hours=1) if pd.notnull(dt) else dt)


In [70]:
# change 22:59:59 back to 23:00:00
hammersmith_verified['date_time'] = hammersmith_verified['date_time'].apply(
    lambda dt: dt.replace(hour=23, minute=0, second=0) if dt.time() == datetime.strptime('22:59:59', '%H:%M:%S').time() else dt
)

In [71]:
hammersmith_verified.head(30)

,date,time,PM10,NO,NO2,NOXasNO2,O3,PM25,date_time
0,01/01/2021,01:00:00,27.5,20.92937,38.37475,70.466,16.81045,24.4,2021-01-01 00:00:00
1,01/01/2021,02:00:00,61.7,6.0364,31.40525,40.66093,17.71516,67.4,2021-01-01 01:00:00
2,01/01/2021,03:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4,2021-01-01 02:00:00
3,01/01/2021,04:00:00,14.2,3.38563,21.6773,26.86853,28.39881,14.4,2021-01-01 03:00:00
4,01/01/2021,05:00:00,19.2,3.0349,20.14869,24.80214,29.4715,19.4,2021-01-01 04:00:00
5,01/01/2021,06:00:00,20.8,3.82942,21.27712,27.14881,28.71812,22.4,2021-01-01 05:00:00
6,01/01/2021,07:00:00,10.8,4.33762,17.10154,23.75246,30.36957,17.4,2021-01-01 06:00:00
7,01/01/2021,08:00:00,12.5,4.44499,23.43834,30.25389,26.81722,20.4,2021-01-01 07:00:00
8,01/01/2021,09:00:00,15,5.58665,23.81465,32.38073,26.66255,15.4,2021-01-01 08:00:00
9,01/01/2021,10:00:00,20.8,6.01612,22.81541,32.03999,28.01963,19.4,2021-01-01 09:00:00


In [72]:
hammersmith_verified = hammersmith_verified.drop(columns = ['date', 'time'])
hammersmith_verified.head()

,PM10,NO,NO2,NOXasNO2,O3,PM25,date_time
0,27.5,20.92937,38.37475,70.466,16.81045,24.4,2021-01-01 00:00:00
1,61.7,6.0364,31.40525,40.66093,17.71516,67.4,2021-01-01 01:00:00
2,25.8,3.36416,21.27606,26.43437,30.83357,24.4,2021-01-01 02:00:00
3,14.2,3.38563,21.6773,26.86853,28.39881,14.4,2021-01-01 03:00:00
4,19.2,3.0349,20.14869,24.80214,29.4715,19.4,2021-01-01 04:00:00


## Dealing with nan values

In [73]:
hammersmith_verified.describe(include="all")

,PM10,NO,NO2,NOXasNO2,O3,PM25,date_time
count,22882,24153,24142.0000,24142,22861,23385.0,26280
unique,281,24117,24125.0000,24131,22269,246.0,NaN
top,14.2,90.3915,13.6622,23.25087,19.957,6.0,NaN
freq,766,2,2.0000,2,5,935.0,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-02 11:30:00
min,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:00:00
25%,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-01 17:45:00
50%,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-02 11:30:00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-02 05:15:00
max,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31 23:00:00


In [74]:
hammersmith_verified.dtypes

PM10                 object
NO                   object
NO2                  object
NOXasNO2             object
O3                   object
PM25                 object
date_time    datetime64[ns]
dtype: object

In [75]:
hammersmith_verified = hammersmith_verified.astype({
    'PM10': 'float',
    'NO': 'float',
    'NO2': 'float',
    'NOXasNO2': 'float',
    'O3': 'float',
    'PM25': 'float',
    'date_time':'datetime64[ns]'
})

In [76]:
hammersmith_verified.describe()

,PM10,NO,NO2,NOXasNO2,O3,PM25,date_time
count,22882.000000,24153.000000,24142.000000,24142.000000,22861.000000,23385.000000,26280
mean,19.342946,37.298966,42.923208,99.656075,34.943669,9.800855,2022-07-02 11:30:00
min,-3.300000,0.033990,0.510530,1.442470,-0.883850,-6.000000,2021-01-01 00:00:00
25%,10.800000,6.810790,20.779097,32.239455,18.080440,5.000000,2021-10-01 17:45:00
50%,15.800000,18.870750,37.263005,67.329230,34.586130,7.400000,2022-07-02 11:30:00
75%,23.300000,50.240070,59.708610,138.043785,50.295880,12.400000,2023-04-02 05:15:00
max,243.300000,520.730190,300.888520,1014.158120,122.373530,179.000000,2023-12-31 23:00:00
std,13.755722,46.841140,28.091629,94.418977,20.932881,8.973716,NaN


In [77]:
print(hammersmith_verified.dtypes)
print(hammersmith_verified.columns)

PM10                float64
NO                  float64
NO2                 float64
NOXasNO2            float64
O3                  float64
PM25                float64
date_time    datetime64[ns]
dtype: object
Index(['PM10', 'NO', 'NO2', 'NOXasNO2', 'O3', 'PM25', 'date_time'], dtype='object')


In [78]:

hammersmith_verified.isna().sum()


PM10         3398
NO           2127
NO2          2138
NOXasNO2     2138
O3           3419
PM25         2895
date_time       0
dtype: int64

In [79]:
# set DateTimeIndex
hammersmith_verified.set_index('date_time', inplace=True)



In [80]:
# Use interpolation to fill nan values
hammersmith_interpolated = hammersmith_verified.interpolate(method='time')

In [81]:
# Check this has worked
nans = hammersmith_interpolated.isna().sum()
nans

PM10        0
NO          0
NO2         0
NOXasNO2    0
O3          0
PM25        0
dtype: int64

In [82]:
hammersmith_interpolated.head(50)

,PM10,NO,NO2,NOXasNO2,O3,PM25
date_time,,,,,,
2021-01-01 00:00:00,27.5,20.92937,38.37475,70.46600,16.81045,24.4
2021-01-01 01:00:00,61.7,6.03640,31.40525,40.66093,17.71516,67.4
2021-01-01 02:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4
2021-01-01 03:00:00,14.2,3.38563,21.67730,26.86853,28.39881,14.4
2021-01-01 04:00:00,19.2,3.03490,20.14869,24.80214,29.47150,19.4
2021-01-01 05:00:00,20.8,3.82942,21.27712,27.14881,28.71812,22.4
2021-01-01 06:00:00,10.8,4.33762,17.10154,23.75246,30.36957,17.4
2021-01-01 07:00:00,12.5,4.44499,23.43834,30.25389,26.81722,20.4
2021-01-01 08:00:00,15.0,5.58665,23.81465,32.38073,26.66255,15.4


In [83]:
hammersmith_verified.sort_values(by='date_time')

,PM10,NO,NO2,NOXasNO2,O3,PM25
date_time,,,,,,
2021-01-01 00:00:00,27.5,20.92937,38.37475,70.46600,16.81045,24.4
2021-01-01 01:00:00,61.7,6.03640,31.40525,40.66093,17.71516,67.4
2021-01-01 02:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4
2021-01-01 03:00:00,14.2,3.38563,21.67730,26.86853,28.39881,14.4
2021-01-01 04:00:00,19.2,3.03490,20.14869,24.80214,29.47150,19.4
...,...,...,...,...,...,...
2023-12-31 19:00:00,2.5,3.42369,8.45878,13.70836,64.22143,5.0
2023-12-31 20:00:00,6.7,2.02518,8.77049,11.87572,66.02280,4.0
2023-12-31 21:00:00,4.2,2.42359,6.93509,10.65120,65.84692,0.0


In [84]:
# Save changes to original df
hammersmith_verified = hammersmith_interpolated
hammersmith_verified = hammersmith_verified.reset_index()
hammersmith_verified.head()

,date_time,PM10,NO,NO2,NOXasNO2,O3,PM25
0,2021-01-01 00:00:00,27.5,20.92937,38.37475,70.46600,16.81045,24.4
1,2021-01-01 01:00:00,61.7,6.03640,31.40525,40.66093,17.71516,67.4
2,2021-01-01 02:00:00,25.8,3.36416,21.27606,26.43437,30.83357,24.4
3,2021-01-01 03:00:00,14.2,3.38563,21.67730,26.86853,28.39881,14.4
4,2021-01-01 04:00:00,19.2,3.03490,20.14869,24.80214,29.47150,19.4


In [85]:
hammersmith_verified.describe()

,date_time,PM10,NO,NO2,NOXasNO2,O3,PM25
count,26280,26280.000000,26280.000000,26280.000000,26280.000000,26280.000000,26280.000000
mean,2022-07-02 11:30:00,19.259317,36.971536,44.216214,100.515906,36.170903,9.359597
min,2021-01-01 00:00:00,-3.300000,0.033990,0.510530,1.442470,-0.883850,-6.000000
25%,2021-10-01 17:45:00,11.700000,7.431495,22.076120,34.422863,20.126558,5.000000
50%,2022-07-02 11:30:00,15.800000,22.502932,40.156695,76.086045,36.755310,7.000000
75%,2023-04-02 05:15:00,23.300000,47.453326,61.599975,132.159110,51.445307,11.400000
max,2023-12-31 23:00:00,243.300000,520.730190,300.888520,1014.158120,122.373530,179.000000
std,NaN,13.268820,45.057725,27.508152,90.831346,20.408555,8.617085


### Remove any negative/0 values as these are not possible 

In [86]:
hammersmith_verified = hammersmith_verified[(hammersmith_verified.PM10>0)&(hammersmith_verified.O3>0)&(hammersmith_verified.PM25>0)]

In [87]:
hammersmith_verified.describe()

,date_time,PM10,NO,NO2,NOXasNO2,O3,PM25
count,25409,25409.000000,25409.000000,25409.000000,25409.000000,25409.000000,25409.000000
mean,2022-06-26 17:52:53.536935424,19.617463,37.779062,44.987160,102.534984,35.816830,9.678036
min,2021-01-01 00:00:00,0.800000,0.033990,0.510530,1.442470,0.021350,0.250000
25%,2021-09-24 10:00:00,11.700000,7.834780,23.037550,36.083730,19.638870,5.000000
50%,2022-06-27 15:00:00,15.800000,23.400630,41.282740,79.219590,36.329470,7.000000
75%,2023-03-26 14:00:00,23.300000,48.383530,62.265289,134.365200,51.102790,12.000000
max,2023-12-31 23:00:00,243.300000,510.258130,300.888520,1014.158120,122.373530,179.000000
std,NaN,13.251314,45.325678,27.462636,91.261129,20.403083,8.521887


## Save cleaned data to file

In [88]:
path = '../data_clean/hammersmith_verified_clean.csv'
hammersmith_verified.to_csv(path, index=False)